# Function Calling Example

### 0. Import OpenAI API Key
- Firstly, load OpenAI API key from `.env` file
- Please create a .env file in the root directory of this project
and add `OPENAI_API_KEY=sk-*******`
or just replace the value of OPENAI_API_KEY below!

In [1]:
from os import environ
from dotenv import load_dotenv


load_dotenv()
OPENAI_API_KEY = environ["OPENAI_API_KEY"]
assert OPENAI_API_KEY is not None, "Please set OPENAI_API_KEY in .env file"

### 1. Load function call from FunctionCalls class

- `FunctionCalls` class is a collection of functions.
- It contains many pre-defined functions.
- The functions will be parsed into `FunctionCall` objects only once when Python starts.
- You can get `FunctionCall` object by calling `FunctionCalls.get_function_call` method.
- The argument of `FunctionCalls.get_function_call` method is a function to parse.

In [9]:
from typing import Literal
from app.models.function_calling.base import FunctionCall
from app.models.function_calling.functions import FunctionCalls

web_search_function: FunctionCall = FunctionCalls.get_function_call(
    FunctionCalls.web_search
)

# `functions` contains a list of all the functions that can be called
functions: list[FunctionCall] = [web_search_function]

# "auto" will automatically select a function based on the prompt
function_call: FunctionCall | Literal["auto", "none"] = "auto"

# "none" will not call any function
function_call = "none"

# A specific function is forcibly called by passing the function call object.
# NOTE: `function_call` must be in the `functions`,
# if you define `function_call` as a `FunctionCall` object.
function_call = web_search_function

In [3]:
from app.models.completion_models import FunctionCallParsed
from app.utils.function_calling.request import request_function_call

# `messages` contains a list of messages to send
messages: list[dict[str, str]] = [
    {"role": "user", "content": "What is price of AAPL now?"}
]

# `function_call_parsed` is a dictionary containing `name` and `arguments` of the function call
# NOTE:
function_call_parsed: FunctionCallParsed = await request_function_call(
    messages=messages,
    functions=functions,
    function_call=function_call,
    model="gpt-3.5-turbo",
    api_key=OPENAI_API_KEY,
)

print(f"- function name: {function_call_parsed['name']}")
print(f"- function arguments: {function_call_parsed.get('arguments')}")

- function name: web_search
- function arguments: {'query_to_search': 'current price of AAPL stock'}


In [6]:
from typing import Optional
from app.models.completion_models import FunctionCallParsed
from app.utils.function_calling.request import request_function_call

# `messages` contains a list of messages to send
messages: list[dict[str, str]] = [
    {"role": "user", "content": "What is price of AAPL now?"}
]

# `function_call_parsed` is a dictionary containing `name` and `arguments` of the function call
# NOTE: if `function_call` is "none", an error will be raised when requesting a function call.
is_error_raised: bool = False
try:
    function_call_parsed: Optional[
        FunctionCallParsed
    ] = await request_function_call(
        messages=messages,
        functions=functions,
        function_call="none",
        model="gpt-3.5-turbo",
        api_key=OPENAI_API_KEY,
    )
except ValueError:
    is_error_raised = True

assert is_error_raised

### 2. Define a custom function

In [10]:
from typing import Annotated
from app.utils.function_calling.parser import parse_function_call_from_function
from app.utils.function_calling.request import request_function_call


def weather(
    location: Annotated[
        str,
        "The location to get the weather for.",
    ],
    unit: Annotated[
        str,
        ["celsius", "fahrenheit"],
        "The unit of temperature.",
    ],
):
    """Get the weather for a location."""
    pass


messages: list[dict[str, str]] = [
    {"role": "user", "content": "What’s the weather like in Boston?"}
]
function_call: FunctionCall = parse_function_call_from_function(weather)
functions: list[FunctionCall] = [function_call]
function_call_parsed: FunctionCallParsed = await request_function_call(
    messages=messages,
    functions=functions,
    function_call=function_call,
    model="gpt-3.5-turbo",
    api_key=OPENAI_API_KEY,
)
print(f"- function name: {function_call_parsed['name']}")
print(f"- function arguments: {function_call_parsed.get('arguments')}")

- function name: weather
- function arguments: {'location': 'Boston', 'unit': 'celsius'}
